In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout,Dense,ReLU,Softmax
from keras.applications.vgg16 import VGG16

In [3]:
df = pd.read_csv('E:/Documents/GitHub/Express-U/data/preprocessed/canny_labels.csv',index_col=False)
df

,file_paths,labels
0,E:/Documents/GitHub/Express-U/data/preprocesse...,1
1,E:/Documents/GitHub/Express-U/data/preprocesse...,1
2,E:/Documents/GitHub/Express-U/data/preprocesse...,1
3,E:/Documents/GitHub/Express-U/data/preprocesse...,1
4,E:/Documents/GitHub/Express-U/data/preprocesse...,1
...,...,...
41995,E:/Documents/GitHub/Express-U/data/preprocesse...,Z
41996,E:/Documents/GitHub/Express-U/data/preprocesse...,Z
41997,E:/Documents/GitHub/Express-U/data/preprocesse...,Z
41998,E:/Documents/GitHub/Express-U/data/preprocesse...,Z


In [4]:
X_Train,X_Test = train_test_split(df,train_size=0.8,random_state=42,shuffle=True)


In [5]:
Train_image_generator = ImageDataGenerator(rescale=1/255,
                                        height_shift_range=0.1,
                                        width_shift_range=0.1,
                                        horizontal_flip=True,
                                        zoom_range=0.5,
                                        shear_range = 0.5,
                                        brightness_range=[0.6,1.0],
                                        rotation_range=15,
                                        fill_mode = 'nearest',
                                        validation_split=0.1)
Test_image_generator = ImageDataGenerator(rescale=1/255)

In [6]:
Train_Set = Train_image_generator.flow_from_dataframe(dataframe=X_Train,
                                                  x_col="file_paths",
                                                  y_col="labels",
                                                  batch_size=32,
                                                  class_mode="categorical",
                                                  color_mode = 'rgb',
                                                  subset="training")

Validation_Set = Train_image_generator.flow_from_dataframe(dataframe=X_Train,
                                                  x_col="file_paths",
                                                  y_col="labels",
                                                  batch_size=32,
                                                  class_mode="categorical",
                                                  color_mode='rgb',
                                                  subset="validation")


Test_Set = Test_image_generator.flow_from_dataframe(dataframe=X_Test,
                                                  x_col="file_paths",
                                                  y_col="labels",
                                                  batch_size=32,
                                                  class_mode="categorical",
                                                  color_mode='rgb',
                                                  shuffle=False) 

Found 30240 validated image filenames belonging to 35 classes.
Found 3360 validated image filenames belonging to 35 classes.
Found 8400 validated image filenames belonging to 35 classes.


In [7]:
Train_Set.image_shape

(256, 256, 3)

In [8]:
model = VGG16(input_shape=(256,256,3),include_top=False,weights='imagenet')

In [9]:
classifier = model.output#head mode
classifier = Flatten()(classifier)#adding layer of flatten
classifier = Dense(units=256, activation='relu')(classifier)
classifier = Dropout(0.6)(classifier)
classifier = Dense(units=35, activation='softmax')(classifier)

model = Model(inputs = model.input , outputs = classifier)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [11]:
model.input.shape

TensorShape([None, 256, 256, 3])

In [12]:
history = model.fit(Train_Set,epochs=10,steps_per_epoch=(Train_Set.image_shape[0] // 32),validation_steps=(Validation_Set.image_shape[0] // 32),batch_size=32,verbose=1,validation_data=Validation_Set)

Epoch 1/10
8/8 [==============================] - 1041s 134s/step - loss: 5.1574 - accuracy: 0.0312 - val_loss: 3.5557 - val_accuracy: 0.0078
Epoch 2/10
8/8 [==============================] - 1053s 135s/step - loss: 3.6857 - accuracy: 0.0352 - val_loss: 3.5657 - val_accuracy: 0.0391
Epoch 3/10
8/8 [==============================] - 788s 102s/step - loss: 3.5848 - accuracy: 0.0352 - val_loss: 3.5942 - val_accuracy: 0.0234
Epoch 4/10
8/8 [==============================] - 790s 102s/step - loss: 3.5624 - accuracy: 0.0508 - val_loss: 3.5569 - val_accuracy: 0.0117
Epoch 5/10
8/8 [==============================] - 632s 81s/step - loss: 3.5550 - accuracy: 0.0195 - val_loss: 3.5550 - val_accuracy: 0.0352
Epoch 6/10
8/8 [==============================] - 620s 80s/step - loss: 3.5560 - accuracy: 0.0391 - val_loss: 3.5546 - val_accuracy: 0.0312
Epoch 7/10
8/8 [==============================] - 630s 81s/step - loss: 3.5545 - accuracy: 0.0469 - val_loss: 3.5564 - val_accuracy: 0.0234
Epoch 8/10
8/8

In [ ]:
model.evaluate(Test_Set,steps=(Test_Set.image_shape[0] // 32))

In [ ]:
# model.save('model_vgg16.h5')
# model.save_weights('model_vgg16_weights.h5')